In [1]:
from typing import List, Dict, Union

import pandas as pd
import scrapbook as sb

from ni_data_space_analyzer import DataSpaceAnalyzer
from ni_data_space_analyzer import DatasetLoader
from ni_data_space_analyzer.exception import DataSpaceAnalyzerError

Notebook Input Parameters


In [2]:
channels = ""
analysis_options = []

Supported Input analysis options and their output types

In [3]:
supported_analysis = [
    {"id": "min", "type": "scalar"},
    {"id": "max", "type": "scalar"},
    {"id": "mean", "type": "scalar"},
    {"id": "2std", "type": "scalar"},
    {"id": "-2std", "type": "scalar"},
    {"id": "moving_mean", "type": "vector"},
    {"id": "cp", "type": "vector"},
    {"id": "cpk", "type": "vector"},
]

supported_analysis_options = list(map(lambda x: x["id"], supported_analysis))

In [4]:
def validate_analysis_options(analysis_options) -> None:
    analysis_options = list(map(str.strip, analysis_options))

    invalid_options = list(set(analysis_options) - set(supported_analysis_options))

    if invalid_options:
        raise DataSpaceAnalyzerError(
            "The analysis failed because the following options are not supported: {0}.".format(
                ", ".join(invalid_options)
            )
        )

In [5]:
def load_channels() -> List[Dict[str, Union[str, pd.DataFrame]]]:
    data = DatasetLoader().load_dataset(channels)
    return data

In [6]:
def analyze_channel_data(channel_data: pd.DataFrame) -> pd.DataFrame:
    data_space_analyzer = DataSpaceAnalyzer(dataframe=channel_data)

    for option in analysis_options:
        if option == "min":
            data_space_analyzer.compute_min()
        elif option == "max":
            data_space_analyzer.compute_max()
        elif option == "mean":
            data_space_analyzer.compute_mean()
        elif option == "2std":
            data_space_analyzer.compute_2std()
        elif option == "-2std":
            data_space_analyzer.compute_negative_2std()
        elif option == "moving_mean":
            data_space_analyzer.compute_moving_mean()
        elif option == "cp":
            data_space_analyzer.compute_cp()
        elif option == "cpk":
            data_space_analyzer.compute_cpk()

    return data_space_analyzer.generate_analysis_output(
        analysis_options=analysis_options, supported_analysis=supported_analysis
    )

In [7]:
analysis_options = list(map(str.lower, analysis_options))
final_result = []

try:
    validate_analysis_options(analysis_options)
    channels = load_channels()

    for channel in channels:
        channel_name = channel["name"]
        channel_data = channel["data"]

        analysis_results = analyze_channel_data(channel_data)
        
        final_result.append({"plot_label": channel_name, "data": analysis_results})

except DataSpaceAnalyzerError as e:
    raise Exception(e) from None
        
sb.glue("result", final_result)